In [ ]:
import pycolmap
import open3d as o3d
import numpy as np
import os
from pathlib import Path

def reconstruct_and_visualize(image_dir, output_dir="reconstruction_output"):
    """
    使用pycolmap对图片进行三维重建，并用Open3D显示结果
    
    参数:
        image_dir: 图片目录路径
        output_dir: 重建结果输出目录
    """
    
    os.makedirs(output_dir, exist_ok=True)
    
    database_path = os.path.join(output_dir, "database.db")
    
    print("=" * 50)
    print("开始三维重建流程")
    print(f"图片目录: {image_dir}")
    print(f"输出目录: {output_dir}")
    print("=" * 50)
    
    try:
        print("步骤 1/4: 提取图像特征...")
        pycolmap.extract_features(database_path, image_dir)
        print("步骤 2/4: 进行特征匹配...")
        pycolmap.match_exhaustive(database_path)
        
        print("步骤 3/4: 进行稀疏重建...")
        maps = pycolmap.incremental_mapping(
            database_path=database_path,
            image_path=image_dir,
            output_path=output_dir
        )
        
        if not maps:
            print("错误: 重建失败！可能的原因：")
            print("- 图片数量不足")
            print("- 图片质量差或模糊")
            print("- 图片之间重叠区域不足")
            print("- 光照条件变化太大")
            return None
        
        reconstruction = list(maps.values())[0]
        
        print(f"步骤 4/4: 重建完成！")
        print(f"重建统计信息:")
        print(f"  - 注册图像数量: {reconstruction.num_reg_images()}")
        print(f"  - 3D点数量: {reconstruction.num_points3D()}")
        print(f"  - 相机数量: {reconstruction.num_cameras()}")
        
        return reconstruction
        
    except Exception as e:
        print(f"重建过程中出现错误: {e}")
        return None

def create_open3d_pointcloud(reconstruction):
    """
    将pycolmap重建结果转换为Open3D点云
    
    参数:
        reconstruction: pycolmap重建对象
    
    返回:
        open3d.geometry.PointCloud: Open3D点云对象
    """
    pcd = o3d.geometry.PointCloud()
    
    points3D = reconstruction.points3D
    
    if not points3D:
        print("警告: 没有找到3D点")
        return pcd
    
    # 提取点坐标和颜色
    points = []
    colors = []
    
    for point_id, point3D in points3D.items():
        # 坐标
        points.append(point3D.xyz)
        
        # 颜色 (pycolmap中颜色是0-255范围的RGB)
        if hasattr(point3D, 'color') and point3D.color is not None:
            # 归一化到0-1范围
            color_normalized = [c / 255.0 for c in point3D.color]
            colors.append(color_normalized)
        else:
            # 如果没有颜色信息，使用默认颜色
            colors.append([0.7, 0.7, 0.7])  # 灰色
    
    # 设置点云数据
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    
    print(f"Open3D点云创建完成: {len(points)} 个点")
    
    return pcd

def visualize_pointcloud(pcd, window_name="三维重建结果"):
    """
    使用Open3D可视化点云
    
    参数:
        pcd: Open3D点云对象
        window_name: 窗口名称
    """
    
    if len(pcd.points) == 0:
        print("错误: 点云为空，无法可视化")
        return
    
    print("正在打开Open3D可视化窗口...")
    print("操作说明:")
    print("  - 鼠标左键拖拽: 旋转")
    print("  - 鼠标右键拖拽: 平移")
    print("  - 鼠标滚轮: 缩放")
    print("  - 按 'Q' 或 'ESC' 退出")
    
    # 创建可视化窗口
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name=window_name, width=1200, height=800)
    
    # 添加点云到可视化器
    vis.add_geometry(pcd)
    
    # 设置渲染选项
    render_option = vis.get_render_option()
    render_option.point_size = 2.0
    render_option.background_color = np.array([0.1, 0.1, 0.1])  # 深色背景
    
    # 运行可视化
    vis.run()
    vis.destroy_window()

def save_pointcloud(pcd, filename="pointcloud.ply"):
    """
    保存点云到文件
    
    参数:
        pcd: Open3D点云对象
        filename: 输出文件名
    """
    try:
        o3d.io.write_point_cloud(filename, pcd)
        print(f"点云已保存到: {filename}")
    except Exception as e:
        print(f"保存点云时出错: {e}")

def create_camera_frustums(reconstruction, scale=0.1):
    """
    创建相机视锥体用于可视化相机位置和姿态
    
    参数:
        reconstruction: pycolmap重建对象
        scale: 视锥体大小比例
    
    返回:
        list: Open3D几何体列表（线框）
    """
    cameras = []
    
    for image_id, image in reconstruction.images.items():
        # 获取相机参数
        camera = reconstruction.cameras[image.camera_id]
        
        # 创建相机视锥体
        frustum = o3d.geometry.LineSet.create_camera_visualization(
            camera.intrinsics.width, 
            camera.intrinsics.height,
            camera.intrinsics.calibration_matrix(),
            np.linalg.inv(image.projection_matrix()),
            scale=scale
        )
        
        # 设置颜色（例如红色）
        frustum.paint_uniform_color([1.0, 0.0, 0.0])
        cameras.append(frustum)
    
    return cameras

def main():
    """
    主函数
    """
    # 设置图片目录
    image_dir = "e:\computervision"  # 修改为你的图片目录
    
    # 检查图片目录是否存在
    if not os.path.exists(image_dir):
        print(f"错误: 图片目录 '{image_dir}' 不存在！")
        print("请确保：")
        print("1. 图片目录 'tupian' 存在于当前工作目录")
        print("2. 目录中包含足够多的图片（建议20张以上）")
        print("3. 图片格式为常见的jpg、png等")
        return
    
    # 检查目录中是否有图片文件
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif'}
    image_files = [f for f in os.listdir(image_dir) 
                  if os.path.splitext(f)[1].lower() in image_extensions]
    
    if not image_files:
        print(f"错误: 在 '{image_dir}' 目录中没有找到图片文件！")
        print(f"支持的格式: {', '.join(image_extensions)}")
        return
    
    print(f"在 '{image_dir}' 中找到 {len(image_files)} 张图片")
    
    # 输出目录
    output_dir = "reconstruction_output"
    
    # 执行三维重建
    reconstruction = reconstruct_and_visualize(image_dir, output_dir)
    
    if reconstruction is None:
        print("重建失败，程序退出")
        return
    
    # 转换为Open3D点云
    pcd = create_open3d_pointcloud(reconstruction)
    
    if len(pcd.points) == 0:
        print("点云为空，无法继续")
        return
    
    # 保存点云
    save_pointcloud(pcd, os.path.join(output_dir, "reconstructed_pointcloud.ply"))
    
    # 可视化点云
    visualize_pointcloud(pcd)
    
    # 可选：显示带相机位置的点云
    print("\n正在创建带相机位置的可视化...")
    try:
        # 创建相机视锥体
        cameras = create_camera_frustums(reconstruction)
        
        # 合并所有几何体
        geometries = [pcd] + cameras
        
        # 可视化点云和相机
        visualize_pointcloud_geometries(geometries, "带相机位置的重建结果")
        
    except Exception as e:
        print(f"创建相机可视化时出错: {e}")
        print("继续使用普通点云可视化...")

def visualize_pointcloud_geometries(geometries, window_name="三维重建"):
    """
    可视化多个几何体
    """
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name=window_name, width=1200, height=800)
    
    for geometry in geometries:
        vis.add_geometry(geometry)
    render_option = vis.get_render_option()
    render_option.point_size = 2.0
    render_option.background_color = np.array([0.1, 0.1, 0.1])
    render_option.line_width = 2.0
    
    vis.run()
    vis.destroy_window()

if __name__ == "__main__":
    main()

在 'e:\computervision' 中找到 5 张图片
开始三维重建流程
图片目录: e:\computervision
输出目录: reconstruction_output
步骤 1/4: 提取图像特征...
步骤 2/4: 进行特征匹配...
步骤 3/4: 进行稀疏重建...
步骤 4/4: 重建完成！
重建统计信息:
  - 注册图像数量: 5
  - 3D点数量: 2013
  - 相机数量: 1
Open3D点云创建完成: 2013 个点
点云已保存到: reconstruction_output\reconstructed_pointcloud.ply
正在打开Open3D可视化窗口...
操作说明:
  - 鼠标左键拖拽: 旋转
  - 鼠标右键拖拽: 平移
  - 鼠标滚轮: 缩放
  - 按 'Q' 或 'ESC' 退出

正在创建带相机位置的可视化...
创建相机可视化时出错: 'pycolmap._core.Camera' object has no attribute 'intrinsics'
继续使用普通点云可视化...
